In [1]:
import time
import random
import os
import pandas as pd
import csv
import numpy as np
from itertools import chain
from pdp_lib import processing_google_map as proc
from pdp_lib import util
from GA_lib import GA_multi_depot as GA
from GA_lib import operation_multi_depot as operation
from GA_lib import evaluate_multi_depot as evaluate
from statistics import mode

In [53]:
def solve_and_print(filepath, rounds = 2, population_size = 100, generations=2000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.vote_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,k=3)
    DEPOT_NUMBERS = proc.final_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,k=3)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    print(REQ_BY_DEPOTS)
    print('----------------Instance name: ' + str(filepath) + '-----------------------------------')
    print(" processing time --- %s seconds ---" % (time.time() - process_time))

    total_results = []
    for round in range(rounds):
        results_all_depots = []
        distance_all_depots = []
        time_all_depots = []
        for id_depot in range(len(DEPOTS)):
            THIS_DEP_REQS = REQ_BY_DEPOTS[id_depot]
            NODES_OF_THIS_DEPOT = proc.locations_of_this_depot(id_depot, REQ_BY_DEPOTS, LOCATIONS)
            if(len(NODES_OF_THIS_DEPOT)==0):
                results_all_depots.append([])
                distance_all_depots.append(0)
                time_all_depots.append(0)
                continue
            # ############### SOLVING THE PROBLEMS !!!!!!!! ######################################
            pops_create_time = time.time()
            calculation_time = time.time()
            print('#################### Round :' +str(round+1)+' ############################')
            ## Initialize the populations
            populations = []
            for _ in range(population_size):
                chromosome = GA.initialize_Feasible_chromosome(DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS, LoadCapacities)
                populations.append(chromosome)
            # print("Populations creation time --- %s seconds ---" % (time.time() - pops_create_time))
            ## Crossovers and mutate ##
            GA_time = time.time()

            ## Evaluate fitnesses of all populations ##
            best_fitness_so_far = -99999999
            for gen in range(generations):
                fitness_table = []
                for chromosome in populations:
                    fitness_table.append(
                        evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,
                                                   id_depot))
                populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
                populations.pop()
                populations.pop()
                elite1 = populations.pop(0)
                elite2 = populations.pop(0)


                ## Tournament Selection ##
                # id1, id2 = random.randrange(0, len(populations)), random.randrange(0, len(populations))
                id1,id2,id3,id4 = random.sample(range(0, len(populations)), 4)
                fitness1,fitness2,fitness3,fitness4 = fitness_table[id1],fitness_table[id2],fitness_table[id3],fitness_table[id4]
                candidate1,candidate2,candidate3,candidate4 = populations[id1], populations[id2],populations[id3],populations[id4]
                parent1 = operation.tournament_selection(candidate1,candidate2,fitness1,fitness2)
                parent2 = operation.tournament_selection(candidate3,candidate4,fitness3,fitness4)

                ## Crossing over !! ####
                child1, child2 = operation.crossover(DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, parent1, parent2,
                                                     DEMANDS, LoadCapacities, DISTANCES_FROM_DEPOTS,
                                                     DISTANCES_TO_DEPOTS, id_depot, crossoverRate)
                # if(not evaluate.haveEqualNodes(child1,child2,LOCATIONS)):
                #         print('note have Equal nodes, Crossover Bug!!!!!'+ str(gen))
                #         break
                #
                ## Mutation ##
                child1 = operation.mutate(child1, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                child2 = operation.mutate(child2, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                if (not evaluate.haveEqualNodes(child1, child2, NODES_OF_THIS_DEPOT)):
                    print('note have Equal nodes, Mutation Bug!!!!!' + str(gen))
                    break


                ## Evaluate Fitness of the best ones ###
                current_fitness = evaluate.chromosomeFitness(child1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                             DISTANCES_TO_DEPOTS, id_depot)
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(child2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))

                ## Add them back into populations ###
                populations.append(child1)
                populations.append(child2)
                populations.append(elite1)
                populations.append(elite2)

                ### Check break conditions ####
                if (current_fitness > best_fitness_so_far):
                    best_fitness_so_far = current_fitness
                    bestFitGen = gen
                    # print('#### New Best Fitness !! , Best so far is :' + str(10000.0 / best_fitness_so_far) + '#####')
                    # print('#### This Generation: ' + str(gen) + '#######')
                if (gen - bestFitGen >= 1000):
                    # print('#### Break Generation: ' + str(gen) + '#######')
                    break
                # print('############# Round-'+str(round+1) +',DEPOT-'+str(id_depot)+','+ 'Generation:' + str(gen + 1) + ' #########################')
                # print('This Gen Distance :' + str(10000.0 / current_fitness))

            ### Evaluate results of current DEPOT ####
            fitness_table = []
            for chromosome in populations:
                fitness_table.append(
                    evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot))
            populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
            # print("GA time --- %s seconds ---" % (time.time() - GA_time))
            best_chromosome = populations[0]
            computational_time = time.time() - calculation_time
            # print("Total Calculation time --- %s seconds ---" % (computational_time))
            distance = evaluate.chromosomeRoutesDistance(best_chromosome,DISTANCES,DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,id_depot)
            # print('Distances of the best chromosome: ' + str(distance))
            # print(best_chromosome)

            ## Memo the results ##
            results_all_depots.append(best_chromosome)
            distance_all_depots.append(distance)
            time_all_depots.append(computational_time)
        total_results.append((results_all_depots,distance_all_depots,time_all_depots))
        print(results_all_depots)
    return total_results

In [54]:
def mean_results(results):
    solutions = []
    cal_times = []
    distances = []
    for round, round_result in enumerate(results):
        sol = results[round][0]
        dist = results[round][1]
        ct = results[round][2]

        solutions.append(sol)
        distances.append(dist)
        cal_times.append(ct)
    distances = np.array(distances)
    distances = pd.DataFrame(distances)
    distances = np.mean(distances, axis=0)
    cal_times = np.array(cal_times)
    cal_times = pd.DataFrame(cal_times)
    cal_times = np.mean(cal_times, axis=0)
    return solutions,distances, cal_times


In [55]:
def create_worst_case(filepath, rounds = 1, population_size = 100, generations=3000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    
    res = GA.initialize_WorstCase_Chromosome(REQUESTS)
    
    
    dist_worst = 0
    for [idx,_,[p,d]] in res:
        depot = int(DEPOT_NUMBERS[idx]) 
        dist_worst += (DISTANCES_FROM_DEPOTS[depot][p]+DISTANCES[p][d]+DISTANCES_TO_DEPOTS[d][depot])

    return res,dist_worst

# Automated run

In [56]:
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))

[{13: (31, 48), 15: (35, 49), 19: (41, 34)}, {38: (79, 61)}, {0: (3, 66), 1: (6, 59), 2: (7, 33), 3: (9, 12), 4: (15, 43), 5: (16, 11), 6: (18, 42), 7: (19, 53), 8: (24, 2), 9: (26, 68), 10: (28, 71), 11: (29, 69), 12: (30, 23), 14: (32, 4), 16: (37, 22), 17: (39, 72), 18: (40, 13), 20: (44, 78), 21: (45, 25), 22: (46, 70), 23: (47, 50), 24: (51, 5), 25: (52, 21), 26: (54, 36), 27: (55, 27), 28: (56, 38), 29: (57, 1), 31: (62, 20), 32: (63, 77), 33: (64, 65), 34: (67, 14), 36: (75, 17), 37: (76, 8), 39: (80, 10)}, {30: (60, 58)}, {35: (74, 73)}]
----------------Instance name: pdp_instances/GOO/doing/goo01.csv-----------------------------------
 processing time --- 0.3970222473144531 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


...

#################### Round :1 ############################


[[[1, [33, 27, 19], [62, 49, 39, 32, 28, 46]], [0, [23, 20], [38, 33, 45, 40]]], [], [[7, [6, 7, 31, 1, 37, 29, 35, 39], [10, 54, 71, 69, 68, 57, 78, 51, 80, 44, 8, 2, 20, 11, 5, 25]], [1, [2, 5], [3, 7, 63, 9]], [0, [14, 9, 8, 38], [24, 14, 12, 50, 77, 76, 74, 52]], [6, [], []], [5, [25, 13, 24, 26, 0, 10], [23, 47, 43, 13, 75, 58, 48, 17, 1, 59, 60, 73]], [2, [11, 22, 21, 34, 36, 28], [18, 34, 37, 35, 55, 53, 70, 72, 66, 65, 16, 15]], [4, [32, 4, 3], [4, 6, 61, 79, 41, 31]], [3, [16, 15, 18, 17, 12], [27, 22, 26, 42, 29, 36, 30, 21, 19, 64]], [8, [], []]], [], [[0, [30], [56, 67]]]]
#################### Round :2 ############################


#################### Round :2 ############################


#################### Round :2 ############################


[[[1, [33, 27, 19], [62, 49, 39, 32, 28, 46]], [0, [23, 20], [38, 33, 45, 40]]], [], [[2, [], []], [7, [2, 3, 32, 4], [3, 4, 63, 6, 61, 79, 41, 31]], [8, [8, 14, 38, 13, 9], [24, 23, 14, 13, 12, 50, 77, 76, 74, 52]], [3, [21, 39, 36, 35, 10, 0], [68, 80, 35, 44, 20, 17, 1, 70, 72, 60, 59, 15]], [0, [25, 22, 28, 34], [47, 37, 58, 55, 53, 66, 65, 16]], [1, [6, 29, 31, 1, 37, 5], [10, 54, 71, 69, 57, 78, 51, 7, 2, 8, 9, 5]], [9, [15, 12, 17, 18, 7], [26, 22, 29, 36, 30, 19, 11, 21, 25, 64]], [6, [11, 24, 16, 26], [18, 27, 34, 43, 42, 48, 75, 73]]], [], [[0, [30], [56, 67]]]]


# The GOO Instances

# Average list by columns

In [57]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [58]:
# res =results[0]
# SOL,DIST,CT = res[0],res[1],res[2]


In [59]:
# worst_sol,distance_worst = create_worst_case('pdp_instances/GOO/goo12.csv')
# distance_worst

In [60]:
# SOL[2]
# count_num_vehicle([worst_sol])

# Writing to CSV

In [61]:
solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo-vote.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo-vote.csv')

# Average list by columns

In [62]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [63]:

# worst_sol

In [64]:
def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

In [65]:
# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_goo.csv')

[1.0, 1.0, 8.5, 1.0, 1.0]
[3.0, 5.0, 2.0, 0.0, 1.0]
[2.0, 0.0, 6.0, 1.0, 0.0]
[1.0, 5.0, 2.0, 0.0, 0.0]
[2.0, 1.0, 6.0, 0.0, 1.0]
[7.0, 0.0, 2.0, 0.0, 0.0]
[1.0, 0.0, 5.0, 0.0, 2.0]
[2.0, 0.0, 5.5, 0.0, 1.0]
[1.0, 0.0, 5.5, 1.0, 0.0]
[6.0, 0.0, 2.0, 0.0, 1.0]
[1.0, 0.0, 2.0, 0.0, 5.0]
[2.0, 0.0, 7.0, 0.0, 1.0]


In [66]:
# results = [([[[2,[16,19],[41,37,34,22]],[1,[13,4,6],[15,31,43,18,48,42]],[0,[15],[35,49]]],[[1,[39,25,24],[52,51,80,21,5,10]],[0,[38],[79,61]]],[[2,[23,7,21],[19,47,45,50,53,25]],[5,[12,31,18],[62,40,30,23,20,13]],[3,[2,32,14,11,27,10,33],[32,7,4,29,33,63,55,77,69,28,27,64,71,65]],[1,[],[]],[0,[34,9,8,0],[3,26,24,68,67,66,2,14]],[4,[3,5],[9,16,12,11]]],[[0,[30],[60,58]]],[[3,[37,35,36],[74,75,73,76,17,8]],[2,[],[]],[1,[29,28,1],[56,57,38,1,6,59]],
                                                                                                                                                                                                                                                                                                                                                                                       [0,[26,22,20,17],[54,46,44,36,39,70,78,72]]]])]

IndentationError: unexpected indent (<ipython-input-66-06f82f746798>, line 2)

In [45]:
# res = results[0]

In [19]:
# all_results

In [20]:
# 
# num_vehicles = np.array(num_vehicles)
# df_num_vehicles = pd.DataFrame(num_vehicles)
# df_num_vehicles.to_csv('num_veh_goo.csv')

In [ ]:
## Run
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))
    
# count veh
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo-3-vote.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo-2-vote.csv')

def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_3-vote.csv')

[{4: (15, 43), 6: (18, 42), 12: (30, 23), 13: (31, 48), 14: (32, 4), 15: (35, 49), 16: (37, 22), 18: (40, 13), 19: (41, 34), 21: (45, 25), 23: (47, 50), 31: (62, 20)}, {24: (51, 5), 25: (52, 21), 38: (79, 61), 39: (80, 10)}, {0: (3, 66), 1: (6, 59), 2: (7, 33), 3: (9, 12), 5: (16, 11), 7: (19, 53), 8: (24, 2), 9: (26, 68), 10: (28, 71), 11: (29, 69), 27: (55, 27), 32: (63, 77), 33: (64, 65), 34: (67, 14)}, {}, {17: (39, 72), 20: (44, 78), 22: (46, 70), 26: (54, 36), 28: (56, 38), 29: (57, 1), 30: (60, 58), 35: (74, 73), 36: (75, 17), 37: (76, 8)}]
----------------Instance name: pdp_instances/GOO/doing/goo1.csv-----------------------------------
 processing time --- 0.3400437831878662 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


[[[3, [31, 6, 12, 23, 18], [62, 47, 40, 30, 23, 20, 18, 13, 50, 42]], [1, [15], [35, 49]], [0, [], []], [2, [16, 19, 14, 4, 13, 21], [32, 15, 4, 31, 41, 43, 48, 45, 37, 34, 22, 25]]], [[1, [38], [79, 61]], [0, [39, 25, 24], [52, 51, 80, 21, 5, 10]]], [[4, [34, 9, 8, 0], [3, 26, 24, 68, 67, 66, 2, 14]], [3, [1, 7], [6, 19, 59, 53]], [0, [32, 11, 10, 2, 27, 33], [7, 29, 33, 63, 55, 77, 69, 28, 27, 64, 71, 65]], [1, [5, 3], [9, 16, 12, 11]]], [], [[3, [37, 35, 36], [74, 75, 73, 76, 17, 8]], [2, [29, 28], [56, 57, 38, 1]], [1, [30], [60, 58]], [0, [20, 22, 17, 26], [54, 46, 44, 36, 39, 70, 78, 72]]]]
#################### Round :2 ############################


...

#################### Round :4 ############################


[[[2, [31, 23, 18, 12, 6], [62, 47, 40, 30, 23, 20, 18, 13, 50, 42]], [1, [15], [35, 49]], [0, [14, 4, 13, 19, 16, 21], [32, 15, 4, 31, 41, 43, 48, 45, 37, 34, 22, 25]]], [[1, [39, 24, 25], [52, 51, 80, 21, 5, 10]], [0, [38], [79, 61]]], [[2, [5, 3], [9, 16, 12, 11]], [1, [11, 32, 2, 10, 27, 33], [7, 29, 33, 63, 55, 77, 69, 28, 27, 64, 71, 65]], [3, [1, 7], [6, 19, 59, 53]], [0, [34, 9, 8, 0], [3, 26, 24, 68, 67, 66, 2, 14]]], [], [[3, [37, 35, 36], [74, 75, 73, 76, 17, 8]], [2, [30], [60, 58]], [1, [29, 28], [56, 57, 38, 1]], [0, [26, 22, 20, 17], [54, 46, 44, 36, 39, 70, 78, 72]]]]
#################### Round :5 ############################


#################### Round :5 ############################


#################### Round :5 ############################


#################### Round :5 ############################


{(1, 2),
 (3, 35),
 (4, 64),
 (8, 13),
 (12, 24),
 (20, 17),
 (23, 41),
 (25, 16),
 (63, 50),
 (65, 21),
 (66, 46),
 (70, 6),
 (72, 28)}

In [82]:
worst_sol2 = [ x for x in worst_sol if tuple(x[2]) in s]

In [83]:
worst_sol2

[]

In [4]:
filepath = 'pdp_instances/GOO/goo01.csv'

process_time = time.time()
numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
LOCATIONS = data[0]
DEMANDS = data[1]
timeWindows = data[2]
serviceTimes = data[3]
pickupSiblings = data[4]
deliverySiblings = data[5]
requestType = data[6]
REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
DEPOTS = proc.create_depots('real_map/depots.csv')
DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
# DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
# DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
# DEPOT_NUMBERS = proc.vote_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,k=3)
DEPOT_NUMBERS = proc.final_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,k=3)
REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)

In [43]:
def analyze_Goo(filepath):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    DEPOT_NUMBERS = proc.final_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,k=3)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    columns = ['pickup','delivery', 'distance_between_nodes','distance_from_depot_to_pickup','distance_from_delivery_to_depot']
    df = pd.DataFrame(columns=columns)
    lst = []
    for i in range(len(REQUESTS)):
        dep = int(DEPOT_NUMBERS[i])
        req= REQUESTS[i]
        pickup = int(req[0])
        delivery = int(req[1])
        distance = DISTANCES[pickup][delivery]
        dist_from_dep = DISTANCES_FROM_DEPOTS[dep][pickup]
        dist_to_dep = DISTANCES_TO_DEPOTS[delivery][dep] 
        lst.append([pickup,delivery,distance,dist_from_dep,dist_to_dep])
        df = pd.DataFrame(lst, columns=columns)
    num_file = filepath.split('o')[-1].split('.')[0]
    df.to_csv('ana/analyzed'+str(num_file)+'.csv')
    print('wrtote file:' + num_file)
    return df

In [44]:
df = analyze_Goo('pdp_instances/GOO/goo01.csv')

wrtote file:01


In [45]:
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    analyze_Goo(filepath)


wrtote file:01


wrtote file:02


wrtote file:03


wrtote file:04


wrtote file:05


...

wrtote file:08


wrtote file:09


wrtote file:10


wrtote file:11


wrtote file:12
